In [1]:
import datetime, measures, mmi_txt_to_cui, umls_tables_processing, utils
import numpy as np
import os
import pandas as pd

from gensim.models import KeyedVectors, Word2Vec
from gensim.test.utils import datapath

0:07:47


### Constants

In [2]:
PATH_EMBEDDINGS = './Embeddings'

### Extracting all the labels for each CUI of the UMLS vocabulary

In [4]:
dict_conso = umls_tables_processing.cui_strings()

0:01:36


In [5]:
len(dict_conso)

3772450

In [6]:
dict_conso['C1963065']

['Apnea, CTCAE', 'Apnea']

## Building  ***seed_rel***

### Extracting all the CUIs related to COPD (only one hop) and its relationships

In [3]:
copd_dict = umls_tables_processing.concepts_related_to_concept(two_way = True, extract_labels = False )
copd_cuis = list(copd_dict.keys())

Building seed time: 0:02:37


#### Check on the relationships proportions in the list: `number of CUI with particular relations / total number of found CUIs`

In [9]:
_ = umls_tables_processing.concepts_related_to_concept(two_way = True, polishing_rels = True, extract_labels = False )
print(str(sum([1 for i in list(_.values()) if len(i)>0]))+'/'+str(len(_)))

Relation '' discarded 
101/256


#### Check which concepts have the empty relation

In [7]:
_ = umls_tables_processing.concepts_related_to_concept(two_way = True, polishing_rels = False, extract_labels = False )
print(str(sum([1 for i in list(_.values()) if '' in i]))+'/'+str(len(_)))

0:01:10
177/256


### Check the semantic types contained into the *seed_rel*

In [8]:
eval_rel = umls_tables_processing.extracting_stys(copd_cuis)
t = mmi_txt_to_cui.check_sty_mmi(mmi_txt_to_cui.convert_sty_stymmi(eval_rel))
print({k: j for k,j in list(t.items())[:5]})

0:09:41
{'dsyn': (105, 'Disease or Syndrome'), 'ftcn': (25, 'Functional Concept'), 'patf': (12, 'Pathologic Function'), 'hlca': (12, 'Health Care Activity'), 'topp': (11, 'Therapeutic or Preventive Procedure')}


In [4]:
seed_rel, _ = umls_tables_processing.extracting_strings(copd_cuis)

Time for extracting labels: 0:00:00


## Building ***seed_paper***

In [5]:
paper_cuis = mmi_txt_to_cui.mmi_to_cui(sty = True)

0:00:00


### Check the semantic types contained into the *seed_paper*

In [3]:
a = mmi_txt_to_cui.check_sty_mmi(paper_cuis)
print({k: j for k,j in list(a.items())[:5]})

{'qlco': (49, 'Qualitative Concept'), 'fndg': (38, 'Finding'), 'inpr': (35, 'Intellectual Product'), 'qnco': (34, 'Quantitative Concept'), 'ftcn': (30, 'Functional Concept')}


In [6]:
seed_paper, _ = umls_tables_processing.extracting_strings([i[0] for i in paper_cuis])

Time for extracting labels: 0:00:00


In [5]:
seed_paper

{'C0030705': ['Patients',
  'patients',
  'Patient',
  'patient',
  'PATIENT',
  '^Patient',
  '*^patient',
  'PT',
  'Patient (person)',
  'LAY USER/PATIENT'],
 'C0024117': ['Chronic Obstructive Airway Disease',
  'Chronic obstructive airway disease',
  'Chronic obstructive airways disease NOS',
  'Chronic Obstructive Airways Disease',
  'Chronic airway obstruction',
  'Chronic Obstructive Lung Disease',
  'CHRONIC OBSTRUCTIVE LUNG DISEASE',
  'Chronic obstructive lung disease',
  'chronic obstructive lung disease',
  'Chronic obstructive lung disease, NOS',
  'Pulmonary Disease, Chronic Obstructive',
  'CHRONIC OBSTRUCTIVE PULMONARY DISEASE',
  'chronic obstructive pulmonary disease',
  'Chronic obstructive pulmonary disease',
  'PULMONARY DISEASE, CHRONIC OBSTRUCTIVE',
  'Chronic obstructive pulmonary disease NOS',
  'Chronic Obstructive Pulmonary Diseases',
  'Chronic Obstructive Pulmonary Disease',
  'COLD',
  'COPD',
  'COPD NOS',
  'HYPERACTIVE AIRWAY DISEASE',
  'Chronic airflo

### Check how many concepts are shared by the two previous seeds

In [7]:
print('Length seed_rel: ' + str(len(seed_rel)))
print('Length seed_paper: ' + str(len(seed_paper)))

Length seed_rel: 256
Length seed_paper: 399


In [8]:
print('The elements shared by two seeds are: ' + str(len(set(seed_rel.keys()).intersection(set(seed_paper.keys())))))

The elements shared by two seeds are: 18


and they are the following:

In [15]:
umls_tables_processing.extracting_strings(list(set(seed_rel.keys()).intersection(set(seed_paper.keys()))))

Time for extracting labels: 0:00:00


({'C0013990': ['Pathological accumulation of air in tissues',
   'Emphysema',
   'EMPHYSEMA',
   'emphysema',
   'Emphysema NOS',
   'Emphysema, NOS',
   'Emphysema (morphologic abnormality)',
   'Emphysema NOS (morphologic abnormality)',
   'abnormal collection of air in tissues'],
  'C0038895': ['Surgical aspects',
   'operative procedures',
   'surgery',
   'operations',
   'invasive procedures',
   'operative therapy',
   'SURG',
   'SU'],
  'C0010200': ['EZTULA',
   'Hoste',
   'Hoesten',
   'Coughing',
   'COUGHING',
   'coughing',
   'Cough',
   'Coughs',
   'COUGH',
   'Cough, NOS',
   'cough',
   'coughs',
   '[D]Cough',
   'C/O - cough',
   'cough symptom',
   'Cough symptom',
   'Cough symptom NOS',
   'cough symptoms',
   'coughing symptom',
   'Coughing - function',
   'Observation of cough',
   'C/O - cough (context-dependent category)',
   '[D]Cough (context-dependent category)',
   'Cough (finding)',
   'Finding of cough',
   'Finding of cough (finding)',
   'Cough symp

#### Building a union of the two seeds:

In [7]:
seed_union = {**seed_rel, **seed_paper}

In [14]:
len(seed_union) == (len(seed_rel) + len(seed_paper))-18

True

## Building ***seed_paper_lite***

From the implementation, the MetaMap Lite version does not work with API service anymore, and given the bad performance related to the mapping, it has not been upgraded.

In [ ]:
paper_lite_cuis, oov_lite = mmi_txt_to_cui.mmi_lite_freetext(sty = True)

In [ ]:
paper_lite_cuis_ = list(set([i[0] for i in paper_lite_cuis]))

### Check the semantic types contained into *the seed_paper_lite*

In [14]:
c = mmi_txt_to_cui.check_sty_mmi(paper_lite_cuis)
print({k: j for k,j in list(c.items())[:5]})

{'inpr': (26, 'Intellectual Product'), 'qlco': (21, 'Qualitative Concept'), 'fndg': (20, 'Finding'), 'idcn': (16, 'Idea or Concept'), 'topp': (13, 'Therapeutic or Preventive Procedure')}


In [15]:
oov_lite

['[84–88].\n', '[92, 93]. \n']

In [ ]:
seed_paper_lite, _ = umls_tables_processing.extracting_strings(paper_lite_cuis_)

In [ ]:
seed_paper_lite

## Loading Models

### Importing the embeddings file names

In [8]:
cuis = ('/cuis/', [f.name for f in os.scandir(PATH_EMBEDDINGS+'/cuis') if (f.is_file())&(f.name != 'README.md')])
words = ('/words/', [f.name for f in os.scandir(PATH_EMBEDDINGS+'/words') if (f.is_file())&(f.name != 'README.md')])
embeddings = [cuis, words]
embeddings

[('/cuis/',
  ['claims_cuis_hs_300.txt.gz',
   'stanford_cuis_svd_300.txt.gz',
   'cui2vec_pretrained.txt',
   'DeVine_etal_200.txt']),
 ('/words/',
  ['PMC-w2v.bin',
   'Health_2.5mreviews.s200.w10.n5.v15.cbow.bin',
   'PubMed-and-PMC-w2v.bin',
   'GoogleNews-vectors-negative300.bin',
   'PubMed-w2v.bin',
   'tweetsvec.txt',
   'wikipedia-pubmed-and-PMC-w2v.bin'])]

### Creating a list of k values to test

In [18]:
ks = [5, 10, 20, 30, 40]
#ks = [5, 10]

### Creating a list of seed tuples

In [18]:
seeds = [('seed_rel', seed_rel), ('seed_paper', seed_paper), ('seed_paper_lite', seed_paper_lite)]

### Creating a dictionary of all the embeddings tested, over the seeds and k values.

In [20]:
a = datetime.datetime.now().replace(microsecond=0)
big_g = {}

for type_emb in embeddings:
    for emb in type_emb[1]:
        model = KeyedVectors.load_word2vec_format(PATH_EMBEDDINGS+type_emb[0]+emb, binary=emb.endswith('.bin'))
        name = os.path.splitext(emb)[0]
        big_g[name] = {}
        print('\n\n The name of embedding is: %s\n' % name)
        for k in ks:
            print('\n k_value: %s\n' % k)
            big_g[name][k] = {}
            for seed in seeds:
                if type_emb[0]=='/cuis/':
                    d = measures.occurred_concept(model, seed[1].keys(), k_most_similar=k)
                    big_g[name][k][seed[0]] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                               measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                               measures.percentage_dcg(d, k=k),
                                               measures.oov(d),
                                               len(seed[1]), []]
                elif type_emb[0]=='/words/':
                    d, new_seed = measures.occurred_labels(model, seed[1], k_most_similar=k)
                    big_g[name][k][seed[0]] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                               measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                               measures.percentage_dcg(d, k=k),
                                               measures.oov(d),
                                               len(seed[1]),
                                               new_seed]
                print('{:s}: pos_dcg: {:.4f}, neg_dcg: {:.4f}, perc_dcg: {:.4f}, oov: {:d}, #seed: {:d}\n'.
                      format(seed[0],
                             big_g[name][k][seed[0]][0],
                             big_g[name][k][seed[0]][1],
                             big_g[name][k][seed[0]][2],
                             big_g[name][k][seed[0]][3],
                             big_g[name][k][seed[0]][4]))
print(datetime.datetime.now().replace(microsecond=0)-a)        



 The name of embedding is: claims_cuis_hs_300.txt


 k_value: 5

seed_rel: pos_dcg: 0.0192, neg_dcg: 0.9808, perc_dcg: 0.0156, oov: 209, #seed: 256

seed_paper: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 389, #seed: 399

seed_paper_lite: pos_dcg: 0.0011, neg_dcg: 0.9989, perc_dcg: 0.0013, oov: 154, #seed: 157


 k_value: 10

seed_rel: pos_dcg: 0.0141, neg_dcg: 0.9859, perc_dcg: 0.0102, oov: 209, #seed: 256

seed_paper: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 389, #seed: 399

seed_paper_lite: pos_dcg: 0.0007, neg_dcg: 0.9993, perc_dcg: 0.0006, oov: 154, #seed: 157


 k_value: 20

seed_rel: pos_dcg: 0.0114, neg_dcg: 0.9886, perc_dcg: 0.0084, oov: 209, #seed: 256

seed_paper: pos_dcg: 0.0002, neg_dcg: 0.9998, perc_dcg: 0.0003, oov: 389, #seed: 399

seed_paper_lite: pos_dcg: 0.0005, neg_dcg: 0.9995, perc_dcg: 0.0003, oov: 154, #seed: 157


 k_value: 30

seed_rel: pos_dcg: 0.0106, neg_dcg: 0.9894, perc_dcg: 0.0082, oov: 209, #seed: 256

seed_paper: pos_dcg: 

/home/salvatore/venv/lib/python3.8/site-packages/gensim/models/keyedvectors.py:2389: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)




 The name of embedding is: cui2vec_pretrained


 k_value: 5

seed_rel: pos_dcg: 0.0448, neg_dcg: 0.9552, perc_dcg: 0.0406, oov: 120, #seed: 256

seed_paper: pos_dcg: 0.0554, neg_dcg: 0.9446, perc_dcg: 0.0546, oov: 214, #seed: 399

seed_paper_lite: pos_dcg: 0.0156, neg_dcg: 0.9844, perc_dcg: 0.0166, oov: 102, #seed: 157


 k_value: 10

seed_rel: pos_dcg: 0.0400, neg_dcg: 0.9600, perc_dcg: 0.0359, oov: 120, #seed: 256

seed_paper: pos_dcg: 0.0481, neg_dcg: 0.9519, perc_dcg: 0.0446, oov: 214, #seed: 399

seed_paper_lite: pos_dcg: 0.0128, neg_dcg: 0.9872, perc_dcg: 0.0121, oov: 102, #seed: 157


 k_value: 20

seed_rel: pos_dcg: 0.0350, neg_dcg: 0.9650, perc_dcg: 0.0309, oov: 120, #seed: 256

seed_paper: pos_dcg: 0.0390, neg_dcg: 0.9610, perc_dcg: 0.0335, oov: 214, #seed: 399

seed_paper_lite: pos_dcg: 0.0098, neg_dcg: 0.9902, perc_dcg: 0.0083, oov: 102, #seed: 157


 k_value: 30

seed_rel: pos_dcg: 0.0332, neg_dcg: 0.9668, perc_dcg: 0.0296, oov: 120, #seed: 256

seed_paper: pos_dcg: 0.03

0:03:25
seed_paper: pos_dcg: 0.1509, neg_dcg: 0.8491, perc_dcg: 0.1031, oov: 62, #seed: 399

0:01:06
seed_paper_lite: pos_dcg: 0.1143, neg_dcg: 0.8857, perc_dcg: 0.0742, oov: 20, #seed: 157


 k_value: 30

0:01:05
seed_rel: pos_dcg: 0.0378, neg_dcg: 0.9622, perc_dcg: 0.0242, oov: 146, #seed: 256

0:03:24
seed_paper: pos_dcg: 0.1234, neg_dcg: 0.8766, perc_dcg: 0.0803, oov: 62, #seed: 399

0:01:05
seed_paper_lite: pos_dcg: 0.0908, neg_dcg: 0.9092, perc_dcg: 0.0541, oov: 20, #seed: 157


 k_value: 40

0:01:04
seed_rel: pos_dcg: 0.0322, neg_dcg: 0.9678, perc_dcg: 0.0196, oov: 146, #seed: 256

0:03:25
seed_paper: pos_dcg: 0.1072, neg_dcg: 0.8928, perc_dcg: 0.0680, oov: 62, #seed: 399

0:01:05
seed_paper_lite: pos_dcg: 0.0767, neg_dcg: 0.9233, perc_dcg: 0.0433, oov: 20, #seed: 157



 The name of embedding is: PubMed-w2v


 k_value: 5

0:00:52
seed_rel: pos_dcg: 0.1156, neg_dcg: 0.8844, perc_dcg: 0.0930, oov: 136, #seed: 256

0:02:05
seed_paper: pos_dcg: 0.2924, neg_dcg: 0.7076, perc_dcg: 0.

### Saving the obtained dictionary of all the embeddings tested, over the seeds and k values.

In [23]:
a = datetime.datetime.now().replace(microsecond=0)
utils.inputs_save(big_g, 'Utilities/big_g')
print(datetime.datetime.now().replace(microsecond=0)-a)

0:00:00


### Tabling data of dictionary

In [41]:
def table(big_g):
    df_ks = []
    for k in ks:
        names = list(big_g.keys())
        seeds = list(big_g[name][k].keys())
        header = [np.array([k]*24),
                  np.array([seeds[0],seeds[0],seeds[0],seeds[0],seeds[0], seeds[0],
                            seeds[1],seeds[1],seeds[1],seeds[1],seeds[1], seeds[1],
                            seeds[2],seeds[2],seeds[2],seeds[2],seeds[2], seeds[2],
                            seeds[3],seeds[3],seeds[3],seeds[3],seeds[3], seeds[3]]), 
                  np.array(['pos_dcg', 'neg_dcg', 'perc_dcg', 'iov%', 'oov', '#seed',
                            'pos_dcg', 'neg_dcg', 'perc_dcg', 'iov%', 'oov', '#seed',
                            'pos_dcg', 'neg_dcg', 'perc_dcg', 'iov%', 'oov', '#seed',
                            'pos_dcg', 'neg_dcg', 'perc_dcg', 'iov%', 'oov', '#seed'])] 
        tmp = [ ['%.3f'%(big_g[name][k][seed][0]),
                 '%.3f'%(big_g[name][k][seed][1]),
                 '%.3f'%(big_g[name][k][seed][2]),
                 '%.2f'%(((big_g[name][k][seed][4] - big_g[name][k][seed][3])/(big_g[name][k][seed][4]))*100)+'%',
                 '%d'%(big_g[name][k][seed][3]),
                 '%d'%(big_g[name][k][seed][4])] for name in names for seed in seeds]
        df_ks.append(pd.DataFrame(np.reshape(tmp, (11, 24)), index=np.array(names), columns = header))
    return df_ks

In [42]:
df_ks = table(big_g)

In [43]:
df_ks[0]

5                           \
                                        seed_rel                            
                                         pos_dcg neg_dcg perc_dcg    iov%   
claims_cuis_hs_300.txt                     0.019   0.981    0.016  18.36%   
stanford_cuis_svd_300.txt                  0.000   1.000    0.000   0.00%   
cui2vec_pretrained                         0.045   0.955    0.041  53.12%   
DeVine_etal_200                            0.044   0.956    0.041  46.88%   
PMC-w2v                                    0.103   0.897    0.086  46.48%   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.041   0.959    0.040  29.30%   
PubMed-and-PMC-w2v                         0.123   0.877    0.105  46.88%   
GoogleNews-vectors-negative300             0.085   0.915    0.070  42.97%   
PubMed-w2v                                 0.116   0.884    0.093  46.88%   
tweetsvec                                  0.021   0.979    0.019  24.61%   
wikipedia-pubmed-and-PMC-w2v               0.114   0.886    0.095  46.88%   

                                                                       \
                                                   seed_paper           
                                         oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                   209   256      0.000   1.000   
stanford_cuis_svd_300.txt                256   256      0.000   1.000   
cui2vec_pretrained                       120   256      0.055   0.945   
DeVine_etal_200                          136   256      0.053   0.947   
PMC-w2v                                  137   256      0.275   0.725   
Health_2.5mreviews.s200.w10.n5.v15.cbow  181   256      0.089   0.911   
PubMed-and-PMC-w2v                       136   256      0.292   0.708   
GoogleNews-vectors-negative300           146   256      0.281   0.719   
PubMed-w2v                               136   256      0.292   0.708   
tweetsvec                                193   256      0.035   0.965   
wikipedia-pubmed-and-PMC-w2v             136   256      0.275   0.725   

                                                          ...                  \
                                                          ... seed_paper_lite   
                                        perc_dcg    iov%  ...        perc_dcg   
claims_cuis_hs_300.txt                     0.000   2.51%  ...           0.001   
stanford_cuis_svd_300.txt                  0.000   0.00%  ...           0.000   
cui2vec_pretrained                         0.055  46.37%  ...           0.017   
DeVine_etal_200                            0.048  43.36%  ...           0.014   
PMC-w2v                                    0.245  88.22%  ...           0.167   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.083  48.12%  ...           0.050   
PubMed-and-PMC-w2v                         0.254  88.47%  ...           0.172   
GoogleNews-vectors-negative300             0.242  84.46%  ...           0.191   
PubMed-w2v                                 0.256  88.47%  ...           0.197   
tweetsvec                                  0.030  42.11%  ...           0.013   
wikipedia-pubmed-and-PMC-w2v               0.231  88.47%  ...           0.175   

                                                                               \
                                                           seed_union           
                                           iov%  oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                    1.91%  154   157      0.009   0.991   
stanford_cuis_svd_300.txt                 0.00%  157   157      0.000   1.000   
cui2vec_pretrained                       35.03%  102   157      0.056   0.944   
DeVine_etal_200                          32.48%  106   157      0.052   0.948   
PMC-w2v                                  88.54%   18   157      0.218   0.782   
Health_2.5mreviews.s200.w10.n5.v15.cbow  42.04%   91   157      0.071   0.929   
PubMed-and-PMC-w2v                       88.54%   18   157      0.236   0.764   


In [44]:
df_ks[1]

10                           \
                                        seed_rel                            
                                         pos_dcg neg_dcg perc_dcg    iov%   
claims_cuis_hs_300.txt                     0.014   0.986    0.010  18.36%   
stanford_cuis_svd_300.txt                  0.000   1.000    0.000   0.00%   
cui2vec_pretrained                         0.040   0.960    0.036  53.12%   
DeVine_etal_200                            0.037   0.963    0.032  46.88%   
PMC-w2v                                    0.084   0.916    0.070  46.48%   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.032   0.968    0.028  29.30%   
PubMed-and-PMC-w2v                         0.094   0.906    0.074  46.88%   
GoogleNews-vectors-negative300             0.064   0.936    0.046  42.97%   
PubMed-w2v                                 0.089   0.911    0.068  46.88%   
tweetsvec                                  0.019   0.981    0.016  24.61%   
wikipedia-pubmed-and-PMC-w2v               0.089   0.911    0.070  46.88%   

                                                                       \
                                                   seed_paper           
                                         oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                   209   256      0.000   1.000   
stanford_cuis_svd_300.txt                256   256      0.000   1.000   
cui2vec_pretrained                       120   256      0.048   0.952   
DeVine_etal_200                          136   256      0.040   0.960   
PMC-w2v                                  137   256      0.216   0.784   
Health_2.5mreviews.s200.w10.n5.v15.cbow  181   256      0.072   0.928   
PubMed-and-PMC-w2v                       136   256      0.219   0.781   
GoogleNews-vectors-negative300           146   256      0.208   0.792   
PubMed-w2v                               136   256      0.217   0.783   
tweetsvec                                193   256      0.031   0.969   
wikipedia-pubmed-and-PMC-w2v             136   256      0.211   0.789   

                                                          ...                  \
                                                          ... seed_paper_lite   
                                        perc_dcg    iov%  ...        perc_dcg   
claims_cuis_hs_300.txt                     0.000   2.51%  ...           0.001   
stanford_cuis_svd_300.txt                  0.000   0.00%  ...           0.000   
cui2vec_pretrained                         0.045  46.37%  ...           0.012   
DeVine_etal_200                            0.032  43.36%  ...           0.011   
PMC-w2v                                    0.177  88.22%  ...           0.111   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.062  48.12%  ...           0.039   
PubMed-and-PMC-w2v                         0.170  88.47%  ...           0.108   
GoogleNews-vectors-negative300             0.157  84.46%  ...           0.125   
PubMed-w2v                                 0.169  88.47%  ...           0.125   
tweetsvec                                  0.027  42.11%  ...           0.010   
wikipedia-pubmed-and-PMC-w2v               0.164  88.47%  ...           0.120   

                                                                               \
                                                           seed_union           
                                           iov%  oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                    1.91%  154   157      0.007   0.993   
stanford_cuis_svd_300.txt                 0.00%  157   157      0.000   1.000   
cui2vec_pretrained                       35.03%  102   157      0.049   0.951   
DeVine_etal_200                          32.48%  106   157      0.042   0.958   
PMC-w2v                                  88.54%   18   157      0.173   0.827   
Health_2.5mreviews.s200.w10.n5.v15.cbow  42.04%   91   157      0.058   0.942   
PubMed-and-PMC-w2v                       88.54%   18   157      0.178   0.822   

In [45]:
df_ks[2]

20                           \
                                        seed_rel                            
                                         pos_dcg neg_dcg perc_dcg    iov%   
claims_cuis_hs_300.txt                     0.011   0.989    0.008  18.36%   
stanford_cuis_svd_300.txt                  0.000   1.000    0.000   0.00%   
cui2vec_pretrained                         0.035   0.965    0.031  53.12%   
DeVine_etal_200                            0.034   0.966    0.031  46.88%   
PMC-w2v                                    0.066   0.934    0.052  46.48%   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.026   0.974    0.021  29.30%   
PubMed-and-PMC-w2v                         0.071   0.929    0.052  46.88%   
GoogleNews-vectors-negative300             0.046   0.954    0.031  42.97%   
PubMed-w2v                                 0.066   0.934    0.048  46.88%   
tweetsvec                                  0.017   0.983    0.015  24.61%   
wikipedia-pubmed-and-PMC-w2v               0.067   0.933    0.048  46.88%   

                                                                       \
                                                   seed_paper           
                                         oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                   209   256      0.000   1.000   
stanford_cuis_svd_300.txt                256   256      0.000   1.000   
cui2vec_pretrained                       120   256      0.039   0.961   
DeVine_etal_200                          136   256      0.032   0.968   
PMC-w2v                                  137   256      0.169   0.831   
Health_2.5mreviews.s200.w10.n5.v15.cbow  181   256      0.057   0.943   
PubMed-and-PMC-w2v                       136   256      0.166   0.834   
GoogleNews-vectors-negative300           146   256      0.151   0.849   
PubMed-w2v                               136   256      0.164   0.836   
tweetsvec                                193   256      0.026   0.974   
wikipedia-pubmed-and-PMC-w2v             136   256      0.160   0.840   

                                                          ...                  \
                                                          ... seed_paper_lite   
                                        perc_dcg    iov%  ...        perc_dcg   
claims_cuis_hs_300.txt                     0.000   2.51%  ...           0.000   
stanford_cuis_svd_300.txt                  0.000   0.00%  ...           0.000   
cui2vec_pretrained                         0.033  46.37%  ...           0.008   
DeVine_etal_200                            0.025  43.36%  ...           0.007   
PMC-w2v                                    0.132  88.22%  ...           0.088   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.046  48.12%  ...           0.027   
PubMed-and-PMC-w2v                         0.121  88.47%  ...           0.079   
GoogleNews-vectors-negative300             0.103  84.46%  ...           0.074   
PubMed-w2v                                 0.119  88.47%  ...           0.082   
tweetsvec                                  0.021  42.11%  ...           0.008   
wikipedia-pubmed-and-PMC-w2v               0.118  88.47%  ...           0.075   

                                                                               \
                                                           seed_union           
                                           iov%  oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                    1.91%  154   157      0.006   0.994   
stanford_cuis_svd_300.txt                 0.00%  157   157      0.000   1.000   
cui2vec_pretrained                       35.03%  102   157      0.042   0.958   
DeVine_etal_200                          32.48%  106   157      0.036   0.964   
PMC-w2v                                  88.54%   18   157      0.137   0.863   
Health_2.5mreviews.s200.w10.n5.v15.cbow  42.04%   91   157      0.047   0.953   
PubMed-and-PMC-w2v                       88.54%   18   157      0.136   0.864   

In [46]:
df_ks[3]

30                           \
                                        seed_rel                            
                                         pos_dcg neg_dcg perc_dcg    iov%   
claims_cuis_hs_300.txt                     0.011   0.989    0.008  18.36%   
stanford_cuis_svd_300.txt                  0.000   1.000    0.000   0.00%   
cui2vec_pretrained                         0.033   0.967    0.030  53.12%   
DeVine_etal_200                            0.032   0.968    0.028  46.88%   
PMC-w2v                                    0.056   0.944    0.043  46.48%   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.023   0.977    0.018  29.30%   
PubMed-and-PMC-w2v                         0.059   0.941    0.042  46.88%   
GoogleNews-vectors-negative300             0.038   0.962    0.024  42.97%   
PubMed-w2v                                 0.055   0.945    0.037  46.88%   
tweetsvec                                  0.015   0.985    0.013  24.61%   
wikipedia-pubmed-and-PMC-w2v               0.056   0.944    0.039  46.88%   

                                                                       \
                                                   seed_paper           
                                         oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                   209   256      0.000   1.000   
stanford_cuis_svd_300.txt                256   256      0.000   1.000   
cui2vec_pretrained                       120   256      0.036   0.964   
DeVine_etal_200                          136   256      0.028   0.972   
PMC-w2v                                  137   256      0.144   0.856   
Health_2.5mreviews.s200.w10.n5.v15.cbow  181   256      0.049   0.951   
PubMed-and-PMC-w2v                       136   256      0.139   0.861   
GoogleNews-vectors-negative300           146   256      0.123   0.877   
PubMed-w2v                               136   256      0.137   0.863   
tweetsvec                                193   256      0.023   0.977   
wikipedia-pubmed-and-PMC-w2v             136   256      0.133   0.867   

                                                          ...                  \
                                                          ... seed_paper_lite   
                                        perc_dcg    iov%  ...        perc_dcg   
claims_cuis_hs_300.txt                     0.000   2.51%  ...           0.000   
stanford_cuis_svd_300.txt                  0.000   0.00%  ...           0.000   
cui2vec_pretrained                         0.030  46.37%  ...           0.008   
DeVine_etal_200                            0.021  43.36%  ...           0.006   
PMC-w2v                                    0.110  88.22%  ...           0.065   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.038  48.12%  ...           0.021   
PubMed-and-PMC-w2v                         0.098  88.47%  ...           0.061   
GoogleNews-vectors-negative300             0.080  84.46%  ...           0.054   
PubMed-w2v                                 0.096  88.47%  ...           0.062   
tweetsvec                                  0.019  42.11%  ...           0.006   
wikipedia-pubmed-and-PMC-w2v               0.095  88.47%  ...           0.057   

                                                                               \
                                                           seed_union           
                                           iov%  oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                    1.91%  154   157      0.005   0.995   
stanford_cuis_svd_300.txt                 0.00%  157   157      0.000   1.000   
cui2vec_pretrained                       35.03%  102   157      0.039   0.961   
DeVine_etal_200                          32.48%  106   157      0.032   0.968   
PMC-w2v                                  88.54%   18   157      0.117   0.883   
Health_2.5mreviews.s200.w10.n5.v15.cbow  42.04%   91   157      0.041   0.959   
PubMed-and-PMC-w2v                       88.54%   18   157      0.114   0.886   

In [47]:
df_ks[4]

40                           \
                                        seed_rel                            
                                         pos_dcg neg_dcg perc_dcg    iov%   
claims_cuis_hs_300.txt                     0.010   0.990    0.008  18.36%   
stanford_cuis_svd_300.txt                  0.000   1.000    0.000   0.00%   
cui2vec_pretrained                         0.032   0.968    0.029  53.12%   
DeVine_etal_200                            0.029   0.971    0.026  46.88%   
PMC-w2v                                    0.048   0.952    0.036  46.48%   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.020   0.980    0.016  29.30%   
PubMed-and-PMC-w2v                         0.052   0.948    0.035  46.88%   
GoogleNews-vectors-negative300             0.032   0.968    0.020  42.97%   
PubMed-w2v                                 0.048   0.952    0.032  46.88%   
tweetsvec                                  0.014   0.986    0.012  24.61%   
wikipedia-pubmed-and-PMC-w2v               0.049   0.951    0.034  46.88%   

                                                                       \
                                                   seed_paper           
                                         oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                   209   256      0.000   1.000   
stanford_cuis_svd_300.txt                256   256      0.000   1.000   
cui2vec_pretrained                       120   256      0.032   0.968   
DeVine_etal_200                          136   256      0.025   0.975   
PMC-w2v                                  137   256      0.130   0.870   
Health_2.5mreviews.s200.w10.n5.v15.cbow  181   256      0.043   0.957   
PubMed-and-PMC-w2v                       136   256      0.122   0.878   
GoogleNews-vectors-negative300           146   256      0.107   0.893   
PubMed-w2v                               136   256      0.120   0.880   
tweetsvec                                193   256      0.021   0.979   
wikipedia-pubmed-and-PMC-w2v             136   256      0.118   0.882   

                                                          ...                  \
                                                          ... seed_paper_lite   
                                        perc_dcg    iov%  ...        perc_dcg   
claims_cuis_hs_300.txt                     0.000   2.51%  ...           0.000   
stanford_cuis_svd_300.txt                  0.000   0.00%  ...           0.000   
cui2vec_pretrained                         0.027  46.37%  ...           0.007   
DeVine_etal_200                            0.019  43.36%  ...           0.005   
PMC-w2v                                    0.098  88.22%  ...           0.054   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.033  48.12%  ...           0.018   
PubMed-and-PMC-w2v                         0.084  88.47%  ...           0.050   
GoogleNews-vectors-negative300             0.068  84.46%  ...           0.043   
PubMed-w2v                                 0.083  88.47%  ...           0.053   
tweetsvec                                  0.018  42.11%  ...           0.006   
wikipedia-pubmed-and-PMC-w2v               0.083  88.47%  ...           0.045   

                                                                               \
                                                           seed_union           
                                           iov%  oov #seed    pos_dcg neg_dcg   
claims_cuis_hs_300.txt                    1.91%  154   157      0.005   0.995   
stanford_cuis_svd_300.txt                 0.00%  157   157      0.000   1.000   
cui2vec_pretrained                       35.03%  102   157      0.036   0.964   
DeVine_etal_200                          32.48%  106   157      0.029   0.971   
PMC-w2v                                  88.54%   18   157      0.105   0.895   
Health_2.5mreviews.s200.w10.n5.v15.cbow  42.04%   91   157      0.036   0.964   
PubMed-and-PMC-w2v                       88.54%   18   157      0.100   0.900   

### Creating a dictionary of all the embeddings tested, over the max k value, corresponding to the number of IoV elements.

In [19]:
a = datetime.datetime.now().replace(microsecond=0)
big_k = {}

for type_emb in embeddings:
    for emb in type_emb[1]:
        model = KeyedVectors.load_word2vec_format(PATH_EMBEDDINGS+type_emb[0]+emb, binary=emb.endswith('.bin'))
        name = os.path.splitext(emb)[0]
        big_k[name] = {}
        print('\n\n The name of embedding is: %s\n' % name)
        for seed in seeds:
            if type_emb[0]=='/cuis/':
                Vemb = utils.extract_w2v_vocab(model)
                k_iov = len(list(set(Vemb).intersection(set(seed[1].keys()))))
                if k_iov <=0:
                    k_iov = 1
                
                d = measures.occurred_concept(model, seed[1].keys(), k_most_similar=k_iov)
                big_k[name][seed[0]] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                        measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                        measures.percentage_dcg(d, k=k_iov),
                                        k_iov,
                                        measures.oov(d),
                                        len(seed[1]), []]
            
            elif type_emb[0]=='/words/':
                Vemb = utils.extract_w2v_vocab(model)                
                processed_seed = umls_tables_processing.discarding_labels_oov(Vemb, seed[1])
                k_iov = sum([1 for k,v in processed_seed.items() if len(v)>0])
                if k_iov <= 0:
                    k_iov = 1
                
                d, _ = measures.occurred_labels(model, processed_seed, k_most_similar=k_iov)
                big_k[name][seed[0]] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                        measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                        measures.percentage_dcg(d, k=k_iov),
                                        k_iov,
                                        measures.oov(d),
                                        len(seed[1]), []]
            print('{:s}: pos_dcg: {:.4f}, neg_dcg: {:.4f}, perc_dcg: {:.4f}, iov/k-NN: {:d}, oov: {:d}, #seed: {:d}\n'.
                  format(seed[0],
                         big_k[name][seed[0]][0],
                         big_k[name][seed[0]][1],
                         big_k[name][seed[0]][2],
                         big_k[name][seed[0]][3],
                         big_k[name][seed[0]][4],
                         big_k[name][seed[0]][5]))
print(datetime.datetime.now().replace(microsecond=0)-a)

del big_k['stanford_cuis_svd_300.txt']



 The name of embedding is: claims_cuis_hs_300.txt

seed_rel: pos_dcg: 0.0097, neg_dcg: 0.9903, perc_dcg: 0.0078, iov/k-NN: 47, oov: 209, #seed: 256

seed_paper: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, iov/k-NN: 10, oov: 389, #seed: 399

seed_paper_lite: pos_dcg: 0.0015, neg_dcg: 0.9985, perc_dcg: 0.0021, iov/k-NN: 3, oov: 154, #seed: 157



 The name of embedding is: stanford_cuis_svd_300.txt

seed_rel: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, iov/k-NN: 1, oov: 256, #seed: 256

seed_paper: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, iov/k-NN: 1, oov: 399, #seed: 399

seed_paper_lite: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, iov/k-NN: 1, oov: 157, #seed: 157



/home/salvatore/venv/lib/python3.8/site-packages/gensim/models/keyedvectors.py:2389: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)




 The name of embedding is: cui2vec_pretrained

seed_rel: pos_dcg: 0.0229, neg_dcg: 0.9771, perc_dcg: 0.0198, iov/k-NN: 136, oov: 120, #seed: 256

seed_paper: pos_dcg: 0.0192, neg_dcg: 0.9808, perc_dcg: 0.0155, iov/k-NN: 185, oov: 214, #seed: 399

seed_paper_lite: pos_dcg: 0.0074, neg_dcg: 0.9926, perc_dcg: 0.0060, iov/k-NN: 55, oov: 102, #seed: 157



 The name of embedding is: DeVine_etal_200

seed_rel: pos_dcg: 0.0210, neg_dcg: 0.9790, perc_dcg: 0.0177, iov/k-NN: 120, oov: 136, #seed: 256

seed_paper: pos_dcg: 0.0137, neg_dcg: 0.9863, perc_dcg: 0.0099, iov/k-NN: 173, oov: 226, #seed: 399

seed_paper_lite: pos_dcg: 0.0072, neg_dcg: 0.9928, perc_dcg: 0.0042, iov/k-NN: 51, oov: 106, #seed: 157



 The name of embedding is: PMC-w2v

0:00:02
0:01:29
seed_rel: pos_dcg: 0.0294, neg_dcg: 0.9706, perc_dcg: 0.0205, iov/k-NN: 119, oov: 137, #seed: 256

0:00:02
0:04:02
seed_paper: pos_dcg: 0.0565, neg_dcg: 0.9435, perc_dcg: 0.0452, iov/k-NN: 352, oov: 47, #seed: 399

0:00:01
0:01:15
seed_paper

In [16]:
df_kmax = []
names = list(big_k.keys())
seeds = list(big_k[name].keys())
header = [np.array([seeds[0]+' #256',seeds[0]+' #256', seeds[0]+' #256', seeds[0]+' #256', seeds[0]+' #256',
                    seeds[1]+' #399',seeds[1]+' #399', seeds[1]+' #399', seeds[1]+' #399', seeds[1]+' #399',
                    seeds[2]+' #157',seeds[2]+' #157', seeds[2]+' #157', seeds[2]+' #157', seeds[2]+' #157',
                    seeds[3]+' #637',seeds[3]+' #637', seeds[3]+' #637', seeds[3]+' #637', seeds[3]+' #637']),
          np.array(['pos_dcg', 'neg_dcg', 'perc_dcg', 'iov/k-NN', 'oov',
                    'pos_dcg', 'neg_dcg', 'perc_dcg', 'iov/k-NN', 'oov',
                    'pos_dcg', 'neg_dcg', 'perc_dcg', 'iov/k-NN', 'oov',
                    'pos_dcg', 'neg_dcg', 'perc_dcg', 'iov/k-NN', 'oov'])] 

tmp = [ ['%.3f'%(big_k[name][seed][0]),
         '%.3f'%(big_k[name][seed][1]),
         '%.3f'%(big_k[name][seed][2]),
         '%d'%(big_k[name][seed][3]),
         '%d'%(big_k[name][seed][4])] for name in names for seed in seeds]
df_kmax.append(pd.DataFrame(np.reshape(tmp, (10, 20)), index=np.array(names), columns = header))

In [17]:
df_kmax[0]

seed_rel #256                   \
                                              pos_dcg neg_dcg perc_dcg   
claims_cuis_hs_300.txt                          0.010   0.990    0.008   
cui2vec_pretrained                              0.023   0.977    0.020   
DeVine_etal_200                                 0.021   0.979    0.018   
PMC-w2v                                         0.029   0.971    0.021   
Health_2.5mreviews.s200.w10.n5.v15.cbow         0.016   0.984    0.012   
PubMed-and-PMC-w2v                              0.029   0.971    0.019   
GoogleNews-vectors-negative300                  0.019   0.981    0.012   
PubMed-w2v                                      0.028   0.972    0.018   
tweetsvec                                       0.013   0.987    0.012   
wikipedia-pubmed-and-PMC-w2v                    0.028   0.972    0.019   

                                                      seed_paper #399          \
                                        iov/k-NN  oov         pos_dcg neg_dcg   
claims_cuis_hs_300.txt                        47  209           0.000   1.000   
cui2vec_pretrained                           136  120           0.019   0.981   
DeVine_etal_200                              120  136           0.014   0.986   
PMC-w2v                                      119  137           0.056   0.944   
Health_2.5mreviews.s200.w10.n5.v15.cbow       75  181           0.024   0.976   
PubMed-and-PMC-w2v                           120  136           0.045   0.955   
GoogleNews-vectors-negative300               110  146           0.033   0.967   
PubMed-w2v                                   120  136           0.042   0.958   
tweetsvec                                     63  193           0.013   0.987   
wikipedia-pubmed-and-PMC-w2v                 120  136           0.042   0.958   

                                                                \
                                        perc_dcg iov/k-NN  oov   
claims_cuis_hs_300.txt                     0.000       10  389   
cui2vec_pretrained                         0.015      185  214   
DeVine_etal_200                            0.010      173  226   
PMC-w2v                                    0.045      352   47   
Health_2.5mreviews.s200.w10.n5.v15.cbow    0.019      192  207   
PubMed-and-PMC-w2v                         0.032      353   46   
GoogleNews-vectors-negative300             0.019      337   62   
PubMed-w2v                                 0.028      353   46   
tweetsvec                                  0.011      168  231   
wikipedia-pubmed-and-PMC-w2v               0.029      353   46   

                                        seed_paper_lite #157                   \
                                                     pos_dcg neg_dcg perc_dcg   
claims_cuis_hs_300.txt                                 0.001   0.999    0.002   
cui2vec_pretrained                                     0.007   0.993    0.006   
DeVine_etal_200                                        0.007   0.993    0.004   
PMC-w2v                                                0.041   0.959    0.024   
Health_2.5mreviews.s200.w10.n5.v15.cbow                0.020   0.980    0.013   
PubMed-and-PMC-w2v                                     0.040   0.960    0.022   
GoogleNews-vectors-negative300                         0.035   0.965    0.016   
PubMed-w2v                                             0.043   0.957    0.022   
tweetsvec                                              0.006   0.994    0.005   
wikipedia-pubmed-and-PMC-w2v                           0.037   0.963    0.019   

                                                      seed_union #637          \
                                        iov/k-NN  oov         pos_dcg neg_dcg   
claims_cuis_hs_300.txt                         3  154           0.005   0.995   
cui2vec_pretrained                            55  102           0.020   0.980   
DeVine_etal_200                               51  106           0.014   0.986   
PMC-w2v

New experimentation with the union of `seed_paper` and `seed_rel` is proposed:

In [24]:
a = datetime.datetime.now().replace(microsecond=0)
big_g = utils.inputs_load('Utilities/big_g')

for type_emb in embeddings:
    for emb in type_emb[1]:
        model = KeyedVectors.load_word2vec_format(PATH_EMBEDDINGS+type_emb[0]+emb, binary=emb.endswith('.bin'))
        name = os.path.splitext(emb)[0]
        print('\n\n The name of embedding is: %s\n' % name)
        for k in ks:
            print('\n k_value: %s\n' % k)
            if type_emb[0]=='/cuis/':
                d = measures.occurred_concept(model, seed_union.keys(), k_most_similar=k)
                big_g[name][k]['seed_union'] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                                measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                                measures.percentage_dcg(d, k=k),
                                                measures.oov(d),
                                                len(seed_union), []]
            elif type_emb[0]=='/words/':
                d, new_seed = measures.occurred_labels(model, seed_union, k_most_similar=k)
                big_g[name][k]['seed_union'] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                                measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k)),
                                                measures.percentage_dcg(d, k=k),
                                                measures.oov(d),
                                                len(seed_union),
                                                new_seed]
            print('{:s}: pos_dcg: {:.4f}, neg_dcg: {:.4f}, perc_dcg: {:.4f}, oov: {:d}, #seed: {:d}\n'.
                  format('seed_union',
                         big_g[name][k]['seed_union'][0],
                         big_g[name][k]['seed_union'][1],
                         big_g[name][k]['seed_union'][2],
                         big_g[name][k]['seed_union'][3],
                         big_g[name][k]['seed_union'][4]))
print(datetime.datetime.now().replace(microsecond=0)-a)        



 The name of embedding is: claims_cuis_hs_300.txt


 k_value: 5

seed_union: pos_dcg: 0.0088, neg_dcg: 0.9912, perc_dcg: 0.0072, oov: 584, #seed: 637


 k_value: 10

seed_union: pos_dcg: 0.0067, neg_dcg: 0.9933, perc_dcg: 0.0050, oov: 584, #seed: 637


 k_value: 20

seed_union: pos_dcg: 0.0057, neg_dcg: 0.9943, perc_dcg: 0.0045, oov: 584, #seed: 637


 k_value: 30

seed_union: pos_dcg: 0.0054, neg_dcg: 0.9946, perc_dcg: 0.0044, oov: 584, #seed: 637


 k_value: 40

seed_union: pos_dcg: 0.0051, neg_dcg: 0.9949, perc_dcg: 0.0043, oov: 584, #seed: 637



 The name of embedding is: stanford_cuis_svd_300.txt


 k_value: 5



/home/salvatore/venv/lib/python3.8/site-packages/gensim/models/keyedvectors.py:2389: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)


seed_union: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 637, #seed: 637


 k_value: 10

seed_union: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 637, #seed: 637


 k_value: 20

seed_union: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 637, #seed: 637


 k_value: 30

seed_union: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 637, #seed: 637


 k_value: 40

seed_union: pos_dcg: 0.0000, neg_dcg: 1.0000, perc_dcg: 0.0000, oov: 637, #seed: 637



 The name of embedding is: cui2vec_pretrained


 k_value: 5

seed_union: pos_dcg: 0.0561, neg_dcg: 0.9439, perc_dcg: 0.0546, oov: 328, #seed: 637


 k_value: 10

seed_union: pos_dcg: 0.0488, neg_dcg: 0.9512, perc_dcg: 0.0451, oov: 328, #seed: 637


 k_value: 20

seed_union: pos_dcg: 0.0417, neg_dcg: 0.9583, perc_dcg: 0.0367, oov: 328, #seed: 637


 k_value: 30

seed_union: pos_dcg: 0.0391, neg_dcg: 0.9609, perc_dcg: 0.0345, oov: 328, #seed: 637


 k_value: 40

seed_union: pos_dcg: 0.0365, neg_dcg: 0.963

Refresh the old stored dictionary

In [48]:
a = datetime.datetime.now().replace(microsecond=0)
utils.inputs_save(big_g, 'Utilities/big_g')
print(datetime.datetime.now().replace(microsecond=0)-a)

0:00:00


The experiment on k_most_similar = max k value is repeated on unified seed

In [13]:
a = datetime.datetime.now().replace(microsecond=0)
big_k = utils.inputs_load('Utilities/big_k')

for type_emb in embeddings:
    for emb in type_emb[1]:
        model = KeyedVectors.load_word2vec_format(PATH_EMBEDDINGS+type_emb[0]+emb, binary=emb.endswith('.bin'))
        name = os.path.splitext(emb)[0]
        print('\n\n The name of embedding is: %s\n' % name)
        if type_emb[0]=='/cuis/':
            if name != 'stanford_cuis_svd_300.txt':
                Vemb = utils.extract_w2v_vocab(model)
                k_iov = len(list(set(Vemb).intersection(set(seed_union.keys()))))
                if k_iov <=0:
                    k_iov = 1
                d = measures.occurred_concept(model, seed_union.keys(), k_most_similar=k_iov)
                big_k[name]['seed_union'] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                             measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                             measures.percentage_dcg(d, k=k_iov),
                                             measures.oov(d),
                                             len(seed_union), []]
        elif type_emb[0]=='/words/':
            Vemb = utils.extract_w2v_vocab(model)                
            processed_seed = umls_tables_processing.discarding_labels_oov(Vemb, seed_union)
            k_iov = sum([1 for k,v in processed_seed.items() if len(v)>0])
            if k_iov <= 0:
                k_iov = 1
            d, new_seed = measures.occurred_labels(model, seed_union, k_most_similar=k_iov)
            big_k[name]['seed_union'] = [measures.pos_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                            measures.neg_dcg(d, normalization = True, norm_fact = measures.max_dcg(k_iov)),
                                            measures.percentage_dcg(d, k=k_iov),
                                            measures.oov(d),
                                            len(seed_union),
                                            new_seed]
        if name != 'stanford_cuis_svd_300.txt':
            print('{:s}: pos_dcg: {:.4f}, neg_dcg: {:.4f}, perc_dcg: {:.4f}, oov: {:d}, #seed: {:d}\n'.
                  format('seed_union',
                         big_k[name]['seed_union'][0],
                         big_k[name]['seed_union'][1],
                         big_k[name]['seed_union'][2],
                         big_k[name]['seed_union'][3],
                         big_k[name]['seed_union'][4]))
print(datetime.datetime.now().replace(microsecond=0)-a)        

0:00:00


 The name of embedding is: claims_cuis_hs_300.txt

seed_union: pos_dcg: 0.0048, neg_dcg: 0.9952, perc_dcg: 0.0041, oov: 584, #seed: 637



 The name of embedding is: stanford_cuis_svd_300.txt



 The name of embedding is: cui2vec_pretrained

seed_union: pos_dcg: 0.0201, neg_dcg: 0.9799, perc_dcg: 0.0171, oov: 328, #seed: 637



 The name of embedding is: DeVine_etal_200

seed_union: pos_dcg: 0.0145, neg_dcg: 0.9855, perc_dcg: 0.0115, oov: 356, #seed: 637



 The name of embedding is: PMC-w2v

Time for discarding labels: 0:00:11
0:09:40
seed_union: pos_dcg: 0.0416, neg_dcg: 0.9584, perc_dcg: 0.0336, oov: 179, #seed: 637



 The name of embedding is: Health_2.5mreviews.s200.w10.n5.v15.cbow

Time for discarding labels: 0:00:00
0:00:55
seed_union: pos_dcg: 0.0191, neg_dcg: 0.9809, perc_dcg: 0.0155, oov: 381, #seed: 637



 The name of embedding is: PubMed-and-PMC-w2v

Time for discarding labels: 0:00:11
0:13:15
seed_union: pos_dcg: 0.0324, neg_dcg: 0.9676, perc_dcg: 0.0233, oov: 

In [18]:
a = datetime.datetime.now().replace(microsecond=0)
utils.inputs_save(big_k, 'Utilities/big_k')
print(datetime.datetime.now().replace(microsecond=0)-a)

0:00:00


### Cardinality of the analyzed embeddings:

In [51]:
a = datetime.datetime.now().replace(microsecond=0)

cardinality_vemb = {}
for type_emb in embeddings:
    for emb in type_emb[1]:
        model = KeyedVectors.load_word2vec_format(PATH_EMBEDDINGS+type_emb[0]+emb, binary=emb.endswith('.bin'))
        name = os.path.splitext(emb)[0]
        vemb = utils.extract_w2v_vocab(model)
        cardinality_vemb[name] = (vemb, len(vemb))
        
utils.inputs_save(cardinality_vemb, 'Utilities/cardinality_vembs')
print(datetime.datetime.now().replace(microsecond=0)-a)

In [58]:
tmp = {k:v[1] for k, v in cardinality_vemb.items()}
tmp

{'claims_cuis_hs_300.txt': 14852,
 'stanford_cuis_svd_300.txt': 22705,
 'cui2vec_pretrained': 109053,
 'DeVine_etal_200': 52102,
 'PMC-w2v': 2515686,
 'Health_2.5mreviews.s200.w10.n5.v15.cbow': 73644,
 'PubMed-and-PMC-w2v': 4087446,
 'GoogleNews-vectors-negative300': 3000000,
 'PubMed-w2v': 2351706,
 'tweetsvec': 26278,
 'wikipedia-pubmed-and-PMC-w2v': 5443656}

In [57]:
utils.inputs_save(tmp, 'Utilities/cardinality_vembs')